In [1]:
from src import *
import torch
from functools import partial
import matplotlib.pyplot as plt

from src import *
torch.autograd.set_detect_anomaly(True) 

In [2]:
"""
llama_size = "7b"
use_base_model = True

if not use_base_model:
    model_name_or_path = f"meta-llama/Llama-2-{llama_size}-chat-hf"
else:
    model_name_or_path = f"meta-llama/Llama-2-{llama_size}-hf"

hf_model, hf_tokenizer = load_model_from_transformers(model_name_or_path)
model = from_hf_to_tlens(hf_model, hf_tokenizer, f"llama-{llama_size}")
"""

'\nllama_size = "7b"\nuse_base_model = True\n\nif not use_base_model:\n    model_name_or_path = f"meta-llama/Llama-2-{llama_size}-chat-hf"\nelse:\n    model_name_or_path = f"meta-llama/Llama-2-{llama_size}-hf"\n\nhf_model, hf_tokenizer = load_model_from_transformers(model_name_or_path)\nmodel = from_hf_to_tlens(hf_model, hf_tokenizer, f"llama-{llama_size}")\n'

In [3]:
from transformer_lens import HookedTransformer

model = HookedTransformer.from_pretrained("gpt2-small")
for param in model.parameters():
    param.requires_grad_(False)

Loaded pretrained model gpt2-small into HookedTransformer


In [4]:
forget_dataset, forget_dataloader = create_dataset("harry_potter", model.tokenizer, batch_size=16, max_seq_length=1024)
retain_dataset, retain_dataloader = create_dataset("owt", model.tokenizer, batch_size=16, max_seq_length=16, num_examples=1000)

/opt/conda/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for Skylion007/openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Skylion007/openwebtext
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Loading dataset shards:   0%|          | 0/80 [00:00<?, ?it/s]

In [5]:
batch = next(iter(forget_dataloader))
model.tokenizer.decode(batch["tokens"][0][batch["mask"][0]])

'Gryffindor, Slytherin, Hufflepuff and Ravenclaw.'

In [6]:
batch = next(iter(retain_dataloader))
model.tokenizer.decode(batch["tokens"][0][batch["mask"][0]])

'Port-au-Prince, Haiti (CNN) -- Earthquake victims, writhing'

In [7]:

with torch.no_grad():
    logits, orig_cache = model.run_with_cache(batch["tokens"])
    del logits
    
    mask = NeuronLevelMask(model).cuda()
    logits, new_cache = model.run_with_cache(batch["tokens"])


In [8]:
batch["mask"][-2]

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True])

In [9]:
model.tokenizer.decode(batch["tokens"][-2][batch["mask"][-2]])

'Get cool in-game extras with amiibo accessories! Just tap to score'

In [10]:
orig_cache["blocks.11.hook_resid_post"] == new_cache["blocks.11.hook_resid_post"]


tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

        [[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

        [[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

In [11]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Wed Apr 17 02:26:08 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          On  | 00000001:00:00.0 Off |                    0 |
| N/A   31C    P0              63W / 300W |   1792MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [12]:
mask = NeuronLevelMask(model).cuda()

for param in model.parameters():
    param.requires_grad_(False)

train_mask(
    model=model,
    mask=mask,
    retain_dataloader=retain_dataloader,
    forget_dataloader=forget_dataloader,
    max_gpu_batch_size=4,
    learning_rate=1e-3
)

  1%|          | 1/100 [00:02<04:36,  2.80s/it]

Retain Loss: 4.088, Forget Loss: 0.397, Reg Loss: 3.795


  2%|▏         | 2/100 [00:05<04:30,  2.76s/it]

Retain Loss: 4.007, Forget Loss: 0.443, Reg Loss: 3.800


  3%|▎         | 3/100 [00:08<04:26,  2.75s/it]

...

Retain Loss: 4.233, Forget Loss: 0.449, Reg Loss: 3.804


  6%|▌         | 6/100 [00:16<04:22,  2.79s/it]

Retain Loss: 4.231, Forget Loss: 0.294, Reg Loss: 3.805


/opt/conda/lib/python3.10/site-packages/torch/autograd/__init__.py:200: UserWarning: Error detected in DivBackward0. Traceback of forward call that caused the error:
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/opt/conda/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
    self._run_once()
  File "/

RuntimeError: Function 'DivBackward0' returned nan values in its 0th output.

In [16]:
mask.masks["mlp_8"].min()

tensor(0.9940, device='cuda:0', grad_fn=<MinBackward1>)